In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

plt.rcParams['font.sans-serif'] = ['Hiragino Sans', 'Arial Unicode MS']
plt.rcParams['figure.figsize'] = (12, 6)
sns.set_palette("husl")


In [6]:
games = pd.read_csv('games.csv', encoding='cp932')
players = pd.read_csv('players.csv', encoding='cp932')
stats_bat = pd.read_csv('stats_bat.csv', encoding='cp932')
stats_pit = pd.read_csv('stats_pit.csv', encoding='cp932')
play_info = pd.read_csv('play_info.csv', encoding='cp932')

print(games.shape)
print(players.shape)
print(stats_bat.shape)
print(stats_pit.shape)
print(play_info.shape)


(4382, 28)
(4218, 25)
(3522, 30)
(1749, 38)
(254361, 208)


In [7]:
games.head()


,game_id,year_id,game_kind_id,game_kind_name,game_date,game_year,game_month,stadium_id,stadium_name,home_team_league_id,...,game_ended_time,game_time,game_delayed_time,attendance,home_team_score,away_team_score,inning,top_bottom_id,top_bottom_name,game_end_status
0,2021000095,30018,1,セ・リーグ公式戦,2021-03-26,2021,3,1,東京ドーム,1,...,21:16:00,03:28:00,NaN,9991.0,8,7,9,2,裏,NaN
1,2021000096,30018,1,セ・リーグ公式戦,2021-03-26,2021,3,2,神宮,1,...,21:08:00,03:36:00,NaN,9984.0,3,4,9,2,裏,NaN
2,2021000097,30018,1,セ・リーグ公式戦,2021-03-26,2021,3,30014,マツダスタジアム,1,...,20:57:00,02:54:00,NaN,16452.0,6,7,9,2,裏,NaN
3,2021000098,30018,2,パ・リーグ公式戦,2021-03-26,2021,3,110,楽天生命パーク,2,...,19:40:00,03:40:00,NaN,14559.0,8,2,9,1,表,NaN
4,2021000099,30018,2,パ・リーグ公式戦,2021-03-26,2021,3,7,メットライフ,2,...,20:15:00,02:30:00,NaN,9599.0,4,3,9,1,表,NaN


In [17]:
pitch_data = play_info[['px', 'pz', 'events']].dropna()

pitch_data = pitch_data[(pitch_data['px'].between(-2, 2)) & 
                        (pitch_data['pz'].between(0.5, 4.5))]

pitch_data['is_hit'] = pitch_data['events'].isin(['single', 'double', 'triple', 'home_run'])

x_bins = np.linspace(-0.83, 0.83, 4)
z_bins = np.linspace(1.5, 3.5, 4)

pitch_data['x_zone'] = pd.cut(pitch_data['px'], bins=x_bins, labels=['左','中','右'])
pitch_data['z_zone'] = pd.cut(pitch_data['pz'], bins=z_bins, labels=['下','中','上'])

zone_stats = pitch_data.groupby(['z_zone', 'x_zone']).agg({
    'is_hit': ['sum', 'count']
})
zone_stats.columns = ['安打数', '投球数']
zone_stats['打率'] = (zone_stats['安打数'] / zone_stats['投球数'] * 100).fillna(0)

heatmap_data = zone_stats['打率'].unstack(fill_value=0)

plt.figure(figsize=(10,10))
sns.heatmap(heatmap_data, annot=True, fmt='.1f', cmap='RdYlGn_r',
            linewidths=2, linecolor='black', square=True,
            cbar_kws={'label': '打率 (%)'}, vmin=0, vmax=40)
plt.title('ストライクゾーン別打率分析', fontsize=18, pad=20)
plt.xlabel('左右位置', fontsize=14)
plt.ylabel('高低位置', fontsize=14)
plt.tight_layout()
plt.show()

print(heatmap_data)


KeyError: "None of [Index(['px', 'pz', 'events'], dtype='object')] are in the [columns]"